## 1. Current Work Scope
### Import library

In [1]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
import spacy
from spacy import displacy
from spacy.tokens import DocBin
import sys, fitz
import re
import pandas as pd
import time
import unicodedata
from tqdm import tqdm

pdfcollection = {}

### Initialize all the needed model

In [2]:
model_md = spacy.load("en_core_web_md")
model_sm = spacy.load("en_core_web_sm")
ruler = model_sm.add_pipe("entity_ruler", before="ner")

### Getting email

In [3]:
def take_email(text):
    text = " ".join(text.split("\n"))
    emailpat = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    try :
        email = re.findall(emailpat, text)[0]
    except:
        email = ''
    return email

### Getting mobile phone

In [4]:
def take_mp(text):
    profDic = {}
    text = " ".join(text.split("\n"))
    mppat = r'(?![0-9]{5}\s[0-9]{5})\+?\d{1,3}[-.\s]?\(?\d{2,3}\)?[-.\s]?\d{2,4}[-.\s]?\d{2,4}[-.\s]?\d{2,4}'
    try :
        mp = re.findall(mppat, text)[0]
    except:
        mp = ''
    return mp

### Getting name

In [5]:
def take_name(text):
    text = re.sub(r'\s+\.', '.', text)
    text_copy = text
    find = False
    for row in text_copy.split("\n"):
        if find == True:
            person = re.sub(r':\s*', '', row)
            person = re.sub(r'^\s*', "", person)
            person = re.sub(r'\s*$', "", person)
            return person
        match = re.search(r'\b(?<!\s)(?!of\s+company)(?:fullnames|fullname|names|name)(?=\s*)(?!\s*[a-z0-9_])', row.lower())
        if match:
            if ":" in row.lower():
                person = re.sub(r'(?i)name\s*:\s*', '', row)
                if person == "":
                    find = True
                else:
                    person = re.sub(r'^\s*', "", person)
                    person = re.sub(r'\s*$', "", person)
                    return person
            if "-" in row.lower():
                person = re.sub(r'(?i)name\s*-\s*', '', row)
                person = re.sub(r'^\s*', "", person)
                person = re.sub(r'\s*$', "", person)
                return person
            else:
                rem = str(row)
                rem = re.sub(r'(?i)(names|fullnames|name|fullname)', '', rem)
                if re.search(r'\s*', rem):
                    find = True
                else:
                    return rem
        
    custom_stopword = ["educational", "objective", "project", "reference",  "experience",  "address"]
    init_len = len(text)
    for sub_str in custom_stopword:
        if sub_str in text.lower():
            text_drop = text[:text.lower().index(sub_str)]
            if len(text_drop) >= 0.01 * init_len:
                text = text_drop
    
    custom_row = ["company", "mother", "father", "blessed", "id", "training"]
    filter_row = []
    for row in text.split("\n"):
        if not any(drop in row.lower() for drop in custom_row):
            filter_row.append(row)
    text = "\n".join(filter_row)
    text = ' '.join([word for word in text.split() if word.lower() not in (stopwords.words('english'))])
    text = " ".join(text.split("\n")).lower()
    
    custom_drop = ("civil", "skill", "permanent", "mail", "email", "add", "academic", "branch", "present", "mechanic", 
                   "engineer", "apply", "programming", "examination", "learning", "university", "employment", "date", 
                   "birth", "place", "succeed", "model", "name", "personal", "profile", "page", "secure", "culture", 
                   "offer", "company", "work", "successfully", "growth", "oriented", "field", "infrastructur", 
                   "qualification", "electronic", "entry", "seeking", "position", "organization", "vitae", "curri", 
                   "cirri", "curi", "instagram", "email", "current", "plot", "resume", "contact", "mobile", "phone", 
                   "no", "nationality", "id", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "block", "correspon", 
                   "passport", "career", "stren")
    text = ' '.join([word for word in text.split() if not word.lower().startswith(custom_drop) and not word.lower().endswith(custom_drop)])
        
    custom_punc = [",", "/", "  ", "-", ":", "(", ")"]
    for punc in custom_punc:
        text = text.replace(punc, "")
    
    text = " ".join(st[0].upper()+st[1:] for st in text.split(" ") if st != "")
    resultnlp = model_md(text)
    k = 0
    for ent in resultnlp.ents:
        if ent.label_ == 'PERSON' and k == 0:
            if len(str(ent.text).split(" ")) > 7:
                person = " ".join(str(ent.text).split(" ")[:5])
            else:
                person = str(ent.text)
            k += 1
    if k == 0:
        resultnlp = model_sm(text)
        k = 0
        for ent in resultnlp.ents:
            if ent.label_ == 'PERSON' and k == 0:
                if len(str(ent.text).split(" ")) > 7:
                    person = " ".join(str(ent.text).split(" ")[:5])
                else:
                    person = str(ent.text)
                k += 1
    if k == 0:
        for ent in resultnlp.ents:
            if ent.label_ == 'ORG' and k == 0:
                person = str(ent.text)
                k += 1
        if k == 0:
            if len(text.split(" ")) < 5:
                person = text
            else:
                person = ""
            
    return person

### Getting skills

In [6]:
def take_skills(text):
    profDic = {}
    text = " ".join(text.split("\n"))
    skills = "jz_skill_patterns.jsonl"
    ruler.from_disk(skills)
    resultnlp = model_sm(text)
    skillsList = []
    for ent in resultnlp.ents:
        if ent.label_ == 'SKILL':
            trait = str(ent.text)
            skillsList.append(trait.upper())
            skillsList = list(set(skillsList))
    return skillsList

### Getting language

In [7]:
def take_lang(text):
    profDic = {}
    doc = fitz.open(pdfpath)
    text = ""
    for page in doc:
        text = text + str(page.get_text())
    text = re.sub(r'[^\x00-\x7F]', '', text)
    text = " ".join(text.split("\n"))
    custom_drop = ["purpose"]
    for drop in custom_drop:
        text = text.lower().replace(drop, "")
    langs = "hh_lang_pattern.jsonl"
    ruler.from_disk(langs)
    resultnlp = model_sm(text)
    langList = []
    for ent in resultnlp.ents:
        if ent.label_ == 'LANG':
            lan = str(ent.text)
            langList.append(lan.upper())
            langList = list(set(langList))
    return langList

### Getting education institution

In [8]:
def take_edu(text):
    profDic = {}
    text = re.sub(r':', ':\n', text)
    init_len = len(text)
    
    custom_mark = ["education", "academic", "scholastic", "institut"]
    for sub_str in custom_mark:
        for row in text.lower().split("\n"):
            if sub_str in row.lower() and len(row.split(" ")) <= 5:
                text = text[text.lower().index(sub_str):]
                break
    
    custom_stopword = ["strength", "area", "expertise", "training", "experience", "project", "reference", "contact", "work", "certificat", "job", "personal", "profile"]
    for sub_str in custom_stopword:
        for row in text.lower().split("\n"):
            if sub_str in row.lower() and len(row.split(" ")) <= 5 and sub_str in text.lower():
                text_stop = text[:text.lower().index(sub_str)]
                if len(text_stop) >= 0.05 * init_len:
                    text = text_stop
    
    custom_punc = [",", "/", "  ", "-", ":", "(", ")"]
    for punc in custom_punc:
        text = text.replace(punc, "")
    
    text = " ".join(st[0].upper()+st[1:] for st in text.split(" ") if st != "")
    edupat = r'\b\s*\w+(?:\s+\w+)*\s*(?i:universit(?:y|é|a(?:s|des)?|ies)|polytechnic|institute|college|academy)\w?\s*\b'
    pos_list = list(re.findall(edupat, text))
    
    filt_list = []
    custom_drop = ["education", "scholastic", "institution", "qualification", "from", 'civil', 'bcom', 'mcom', 'btech', 'mtech', 'phd', 'mba','graduate', 'postgraduate', 'master', 'ssc', 'hsc', 'cbse', 'icse', "engineering", "from", "mechanical", "class"]
    for edu in pos_list:
        for drop in custom_drop:
            if drop in edu.lower():
                edu = edu[edu.lower().index(drop):]
            edu = ' '.join(word for word in edu.split() if not word.lower() == drop.lower())
            filt_list.append(edu)
    
    edu_filt = ""
    for pos in filt_list:
        if len(pos.split(" ")) > 1 and len(pos.split(" ")) < 7:
            edu_filt = pos
    return edu_filt

### Getting summary

In [9]:
def take_sum(text):
    profDic = {}
    text = re.sub(r':', ':\n', text)
    init_len = len(text)
       
    found = False
    custom_stopword = ["strength", "expertise", "training", "scholastics", "qualification", "experience", "education", "project", "reference", "contact", "academic", "work", "certificat", "history", "job", "personal", "profile"]
    custom_mark = ["summary", "about me", "introduction", "synopsis", "abstract", "highlight", "overview", "objective", "purpose"]
    for sub_str in custom_mark:
        for row in text.lower().split("\n"):
            if sub_str in row.lower() and len(row.split(" ")) <= 5 and not any(sw in row.lower() for sw in custom_stopword):
                text = text[text.lower().index(sub_str):]
                found = True
            if found == True:
                break
        if found == True:
            break
    
    for sub_str in custom_stopword:
        for row in text.lower().split("\n"):
            if sub_str in row.lower() and len(row.split(" ")) <= 5 and sub_str in text.lower():
                tex = text[:text.lower().index(sub_str)]
                if len(tex) >= 0.01 * init_len:
                    text = tex
    
    custom_drop = ["purpose", "objective", "overview", "highlight", "abstract", "synopsis", "introduction", "about me", "summary"]
    for sub_str in custom_drop:
        for row in text.lower().split("\n"):
            if sub_str in row.lower() and len(row.split(" ")) <= 5 and sub_str in text.lower():
                tex = text[:text.lower().index(sub_str)]
                if len(tex) >= 0.01 * init_len:
                    text = tex
    
    filter_row = []
    for row in text.split("\n"):
        if not any(drop in row.lower() for drop in custom_drop):
            filter_row.append(row)
    
    summary = " ".join(filter_row)
    return summary

### Getting education degree

In [10]:
def take_degree(text):
    profDic = {}
    text = " ".join(text.split("\n"))
    nlp = spacy.load("output/model-best")
    resultnlp = nlp(text)
    degreeD = ["diploma", "amd", "ama", "str", "ap"]
    degree1 = ["bachelor", "btech",  "sarjana", 'ba', 'bcom', 'be', 'btech', 'bsc', 'bba', 'skom', 'se', 'ssi', 'ssn', 'sikom', 'sip', 'sipol', 'sh', 'spd', 'sked', 'skeb', 'sgz', 'shum', 'spsi']
    degree2 = ["mba", 'master', 'magister', 'msc', 'mm', 'ma', 'msi', 'mtech', 'mkom', 'mcs', 'mpsi', 'mpd', 'drs', 'mmgt']
    degree3 = ['prof', 'phd', 'thd', 'dth', 'edd']
    degreeK = ['apt', 'dr', 'ir', 'mr']
    degreeDList = []
    degree1List = []
    degree2List = []
    degree3List = []
    degreeKList = []
    degreeList = []
    for ent in resultnlp.ents:
        if ent.label_ == 'DEGREE':
            degree = str(ent.text)
        if degree in punctuation:
            continue
        degree_clean = re.sub(r'[^\w\s]', '', degree)
        if degree_clean.lower() in degreeD:
            degreeDList.append(degree.upper())
        elif degree_clean.lower() in degree1:
            degree1List.append(degree.upper())
        elif degree_clean.lower() in degree2:
            degree2List.append(degree.upper())
        elif degree_clean.lower() in degree3:
            degree3List.append(degree.upper())
        elif degree_clean.lower() in degreeK:
            degreeKList.append(degree.upper())
        else:
            degreeList.append(degree)
        degreeList = list(set(degreeList))
        degreeDList = list(set(degreeDList))
        degree1List = list(set(degree1List))
        degree2List = list(set(degree2List))
        degree3List = list(set(degree3List))
        degreeKList = list(set(degreeKList))
    return degreeDList, degree1List, degree2List, degree3List, degreeKList, degreeList

### Getting location

In [11]:
def take_country(text):
    profDic = {}
    text = " ".join(text.split("\n"))
    loc = "hh_country_pattern.jsonl"
    ruler.from_disk(loc)
    resultnlp = model_sm(text)
    loc = ''
    for ent in resultnlp.ents:
        if ent.label_ == 'COUNTRY':
            country = str(ent.text)
            loc = country
            break
    return loc

---
## 2. Demo
### Presentation Demo

In [12]:
for i in range(1, 22):
    profDic = {}
    if i in range(1, 21):
        pdfpath = './../data/' + str(i) + '.pdf'
    else:
        pdfpath = './../data/[ML] CV Reynard Matthew Yaputra.pdf'
    
    doc = fitz.open(pdfpath)
    text = ""
    for page in doc:
        text = text + str(page.get_text())
    text = re.sub(r'[^\x00-\x7F]', '', text)
    
    profDic["PERSON"] = take_name(text)
    profDic["EMAIL"] = take_email(text)
    profDic["MOBILE"] = take_mp(text)
    profDic["SUM"] = take_sum(text)
    profDic["SKILL"] = take_skills(text)
    profDic["LANG"] = take_lang(text)    
    profDic["EDU"] = take_edu(text)
    profDic['DEGREE'] = take_degree(text)
    profDic['LOC'] = take_country(text)
    pdfcollection[i] = profDic

print(pdfcollection)

{1: {'PERSON': 'Mike KisasatiWanaswa', 'EMAIL': 'mikewanaswa@gmail.co', 'MOBILE': '0717 550926', 'SUM': 'To put in use the latest inventions in Telecommunication and Information Technology for a  positive impact in Individuals, Business Enterprises and Corporate Organizations. ', 'SKILL': ['WIMAX', 'P2P', 'WIRELESS', 'BOX', 'SCANNER', 'BIOMETRICS', 'NETWORK MANAGEMENT', 'COMPUTER ENGINEERING', 'SECURITY', 'DESIGN', 'SOFTWARE', 'DEPLOYMENT', 'SUPPORT', 'DATABASE', 'BUSINESS'], 'LANG': ['SWAHILI', 'ENGLISH'], 'EDU': 'Inoorero University', 'DEGREE': (['DIPLOMA'], [], [], [], [], ['March2010', '( M', 'Rollout']), 'LOC': 'Kenya'}, 2: {'PERSON': 'Ardra Prasad', 'EMAIL': 'ardraprasad93@gmail.com', 'MOBILE': '9891423551', 'SUM': 'Seeking entry level positions in a growth oriented company in the field of the  infrastructure construction and consulting  ', 'SKILL': ['PROGRAMMING LANGUAGE', 'ENGINEERING', 'CERTIFICATE', 'TERMINAL', 'C++', 'C'], 'LANG': ['MALAYALAM', 'ENGLISH', 'HINDI'], 'EDU': 'S

In [65]:
print(pdfcollection[19])

{'PERSON': 'Ashutosh Mishra', 'EMAIL': 'ashu.mishra166@yahoo.com', 'MOBILE': '+91 8866544469', 'SUM': 'To contribute in enhancing the performance of the company by giving my optimum input and to make one of the admired company of respective field . ', 'SKILL': ['LANGUAGES', 'FINANCE', 'MOBILE'], 'LANG': ['GUJARATI', 'ENGLISH', 'HINDI'], 'EDU': 'Organized College', 'DEGREE': ([], ['BBA'], ['MBA'], [], [], ['been', 'BSE']), 'LOC': ''}


### Development Demo

In [66]:
for i in range(1, 21):
    profDic = {}
    pdfpath = './../data/[ML] CV Reynard Matthew Yaputra.pdf'
    
    start_time = time.time()
    profDic["PERSON"] = take_name(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 1, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic["EMAIL"] = take_email(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 2, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic["MOBILE"] = take_mp(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 3, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic["SUM"] = take_sum(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 4, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic["SKILL"] = take_skills(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 5, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic["LANG"] = take_lang(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 6, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic["EDU"] = take_edu(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 7, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic['DEGREE'] = take_degree(text)
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print("Algorithm runtime 8, Iteration ", i, ": ", elapsed_time, "seconds")
    
#     start_time = time.time()
    profDic['LOC'] = take_country(text)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Algorithm runtime, Iteration ", i, ": ", elapsed_time, "seconds")

    pdfcollection[i] = profDic
#     print(pdfcollection)
    print()

Algorithm runtime, Iteration  1 :  0.6223301887512207 seconds

Algorithm runtime, Iteration  2 :  0.6560003757476807 seconds

Algorithm runtime, Iteration  3 :  0.6220777034759521 seconds

Algorithm runtime, Iteration  4 :  0.5969491004943848 seconds

Algorithm runtime, Iteration  5 :  0.6090116500854492 seconds

Algorithm runtime, Iteration  6 :  0.6304264068603516 seconds

Algorithm runtime, Iteration  7 :  0.6290009021759033 seconds

Algorithm runtime, Iteration  8 :  0.8141613006591797 seconds

Algorithm runtime, Iteration  9 :  0.7407596111297607 seconds

Algorithm runtime, Iteration  10 :  0.6739535331726074 seconds

Algorithm runtime, Iteration  11 :  0.6806154251098633 seconds

Algorithm runtime, Iteration  12 :  0.8410141468048096 seconds

Algorithm runtime, Iteration  13 :  0.6670818328857422 seconds

Algorithm runtime, Iteration  14 :  0.6959636211395264 seconds

Algorithm runtime, Iteration  15 :  0.789783239364624 seconds

Algorithm runtime, Iteration  16 :  0.677446603775

---
## 3. Additional Python Work
### Structuring training and testing data to get education degree

In [12]:
collective_dict = {'TRAINING_DATA': []}

def structure_training_data(text, kw_list):
    results = []
    entities = []
    
    # search for instances of keywords within the text (ignoring letter case)
    for kw in tqdm(kw_list):
        search = re.finditer(kw, text, flags=re.IGNORECASE)
        
        all_instances = [[m.start(),m.end()] for m in search] 
        
        if len(all_instances)>0:
            for i in all_instances:
                start = i[0]
                end = i[1]
                entities.append((start, end, "DEGREE"))
        else:
            print("No pattern matches found. Keyword:", kw)
            
    if len(entities)>0:
        results = [text, {"entities": entities}]
        collective_dict['TRAINING_DATA'].append(results)
        return

degree = ["diploma", "a.md", "a.ma", "s.tr", "a.p", "b.tech",  "bachelor", 'b.a.', 'b.com', 'b.e.', 'b-tech', 'b.sc.', 
        'bba', 's.kom', "mba", 'master', 'magister', 'm.sc', 'm.m', 'm.a', 'm.si', 'm.tech', 
        'm.kom', 'm.cs', 'prof', 'ph.d', 'apt', 'dr', 'th.d', 'd.th', 's.e', 's.si', 's.sn', 's.i.kom',
        's.i.p', 's.i.pol', 's.h', 's.pd', 's.ked', 's.keb', 's.gz', 's.hum', 's.psi', 'm.psi', 'm.pd', 'm.cs', 'drs', 
        'ir', 'mr', 'm.mgt', 'ed.d']

for i in range(1, 202):
    if i in range(1, 201):
        pdfpath = './../data/' + str(i) + '.pdf'
    else:
        pdfpath = './../data/[ML] CV Reynard Matthew Yaputra.pdf'
    doc = fitz.open(pdfpath)
    text = ""
    for page in doc:
        text = text + str(page.get_text())
    text = re.sub(r'[^\x00-\x7F]', '', text)
    custom_punc = [",", "/", "-", ":", "(", ")", '&']
    for punc in custom_punc:
        text = text.replace(punc, " ")
    structure_training_data(text, degree)
    
TRAIN_DATA = collective_dict['TRAINING_DATA']
collective_dict['TRAINING_DATA'] = []

for i in range(201, 221):
    pdfpath = './../data/' + str(i) + '.pdf'
    doc = fitz.open(pdfpath)
    text = ""
    for page in doc:
        text = text + str(page.get_text())
    text = re.sub(r'[^\x00-\x7F]', '', text)
    for punc in custom_punc:
        text = text.replace(punc, " ")
    structure_training_data(text, degree)

TEST_DATA = collective_dict['TRAINING_DATA']
nlp = spacy.blank('en')

def create_training_set(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []

        # create span objects
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract") 

            if span is None:
                print("Skipping entity.")
            else:
                ents.append(span)
                try:
                    doc.ents = ents
                except:
                    ents.pop()
        doc.ents = ents
        db.add(doc)
    return db

TRAIN_DATA_DOC = create_training_set(TRAIN_DATA)
TRAIN_DATA_DOC.to_disk("./TRAIN_DATA.spacy")
TEST_DATA_DOC = create_training_set(TEST_DATA)
TEST_DATA_DOC.to_disk("./TEST_DATA.spacy")

100%|████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 8574.21it/s]


No pattern matches found. Keyword: a.md
No pattern matches found. Keyword: a.ma
No pattern matches found. Keyword: s.tr
No pattern matches found. Keyword: b.tech
No pattern matches found. Keyword: bachelor
No pattern matches found. Keyword: b.a.
No pattern matches found. Keyword: b.com
No pattern matches found. Keyword: b.e.
No pattern matches found. Keyword: b-tech
No pattern matches found. Keyword: b.sc.
No pattern matches found. Keyword: bba
No pattern matches found. Keyword: s.kom
No pattern matches found. Keyword: master
No pattern matches found. Keyword: magister
No pattern matches found. Keyword: m.sc
No pattern matches found. Keyword: m.si
No pattern matches found. Keyword: m.tech
No pattern matches found. Keyword: m.kom
No pattern matches found. Keyword: m.cs
No pattern matches found. Keyword: ph.d
No pattern matches found. Keyword: apt
No pattern matches found. Keyword: th.d
No pattern matches found. Keyword: d.th
No pattern matches found. Keyword: s.si
No pattern matches fou

100%|██████████████████████████████████████████████████████████████████████████| 51/51 [00:00<?, ?it/s]


No pattern matches found. Keyword: diploma
No pattern matches found. Keyword: a.md
No pattern matches found. Keyword: a.ma
No pattern matches found. Keyword: s.tr
No pattern matches found. Keyword: bachelor
No pattern matches found. Keyword: b.com
No pattern matches found. Keyword: b-tech
No pattern matches found. Keyword: b.sc.
No pattern matches found. Keyword: bba
No pattern matches found. Keyword: s.kom
No pattern matches found. Keyword: mba
No pattern matches found. Keyword: master
No pattern matches found. Keyword: magister
No pattern matches found. Keyword: m.sc
No pattern matches found. Keyword: m.m
No pattern matches found. Keyword: m.si
No pattern matches found. Keyword: m.tech
No pattern matches found. Keyword: m.kom
No pattern matches found. Keyword: m.cs
No pattern matches found. Keyword: ph.d
No pattern matches found. Keyword: apt
No pattern matches found. Keyword: th.d
No pattern matches found. Keyword: d.th
No pattern matches found. Keyword: s.si
No pattern matches foun

100%|███████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 50284.32it/s]


...

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entit

100%|███████████████████████████████████████████████████████████████| 201/201 [00:01<00:00, 155.90it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entit


100%|█████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 238.41it/s]

Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entity.
Skipping entit

### List all the languages, sort it based on alphabet, and generate the content of hh_lang_pattern.jsonl 

In [3]:
list_lang = ['Magahi', 'Latin', 'Manx', 'Yapese', 'Bulgarian', 'Lezghian', 'Syriac', 'Picard', 'Tamil', 
             'Gorontalo', 'Gujarati', 'Morisyen', 'Sena', 'Guarani', 'Finnish', 'Bashkir', 'Ligurian', 
             'Norwegian', 'Bihari', 'Arabic', 'Dargwa', 'Kapampangan', 'Gondi', 'Ulithian', 'Klingon', 
             'Tereno', "N'Ko", 'Krio', 'Marshallese', 'Nyankole', 'Selkup', 'Burmese', 'Slave', 
             'Judeo-Arabic', 'Kotava', 'Shawiya', 'Uighur', 'Western Balochi', 'Kashmiri', 
             'Apache', 'Japanese', 'Swati', 'Mandar', 'Norse', 'South Levantine Arabic', 
             'Root', 'Hebrew', 'Swedish', 'Fiji Hindi', 'German', 'Saho', 'Kurukh', 
             'Chhattisgarhi', 'Fala', 'Zapotec', 'Bavarian', 'Javanese', 'Haitian Creole', 
             'Interlingue', 'Min Nan', 'Swahili', 'Italian', 'Tumbuka', 'Galician', 'Khowar', 
             'Sasak', 'Santali', 'Kabardian', 'Southern Altai', 'Classical Newari', 'Gilaki', 
             'Manipuri', 'Vunjo', 'Jola-Fonyi', 'Kosraean', 'Sotho', 'Pangasinan', 'Comorian', 
             'Lombard', 'Tigrinya', 'Selayar', 'Pedi', 'Hausa', 'Georgian', 'Ido', 'Batak', 
             'Bhojpuri', 'Tsonga', 'Belarusian', 'Nias', 'Western Frisian', 'Komi', 'Gayo', 
             'Swazi', 'Dogri', 'Mazanderani', 'Karakalpak', 'Occitan', 'Shona', 'Aymara', 'Luhya', 
             'Nepali', 'Dutch', 'Frisian', 'Classical Syriac', 'Pothwari', 'Maldivian', 'Dhivehi', 
             'Romanian', 'Mandingo', 'Romansh', 'Angika', 'West Flemish', 'Kumyk', 'Soninke', 'Mon', 
             'North Frisian', 'Nauruan', 'Tlingit', 'Albanian', 'Dimli', 'Adyghe', 'Konkani', 'Maori', 
             'Irish', 'Meru', 'Luo', 'Herero', 'Xinjiang Kumo Xi', 'English', 'Greek', 'Tswana', 'Cornish', 
             'Kwanyama', 'Susu', 'Sinhala', 'Russian', 'Votic', 'Ndebele', 'Hupa', 'Lao', 'Welsh', 'Karelian', 
             'Grebo', 'Tunica', 'Aramaic', 'Kirghiz', 'Kom', 'Kirundi', 'Church Slavic', 'Tsakhur', 'Rombo', 
             'Akkadian', 'Zarma', 'Upper Sorbian', 'Moksha', 'Bemba', 'Icelandic', 'Serbian', 'Armenian', 
             'Classical Tibetan', 'Iban', 'Indonesian', 'Luxembourgish', 'Hiri Motu', 'Oriya', 'Gikuyu', 
             'Cherokee', 'Hawaiian', 'Mentawai', 'Kalenjin', 'Piedmontese', 'Pohnpeian', 'Interlingua', 
             'Punjabi', 'Kachin', 'Newari', 'Tulu', 'Pampanga', 'Eastern Tamang', 'Sicilian', 'Chechen', 
             'Chittagonian', 'Hmong', 'Dzongkha', 'Tokelau', 'Sango', 'Lule Sami', 'Nauru', 'Estonian', 
             'Erzya', 'Guaraní', 'Bambara', 'Portuguese', 'Veps', 'Kamba', 'Brahui', 'Samaritan Aramaic', 
             'Fula', 'Wolof', 'Samburu', 'Kannada', 'Assamese', 'Esperanto', 'Fijian', 'Gbaya', 'Balinese', 
             'Eastern Punjabi', 'Catalan', 'Palauan', 'Ganda', 'Nama', 'Pali', 'Chamic', 'Fang', 'Min Dong', 
             'Fon', 'Dinka', 'Chamorro', 'Extremaduran', 'Uzbek', 'Limburgish', 'Bosnian', 'Xipaya', 'Tachelhit', 
             'Friulian', 'Ladino', 'Malayalam', 'Madurese', 'Siswati', 'Mundang', 'Toda', 'Divehi', 'Kutenai', 
             'Udmurt', 'Ojibwa', 'Berber', 'Choctaw', 'Twi', 'Tamashek', 'Kinyarwanda', 'Tok Pisin', 'Urdu', 
             'Luba-Katanga', 'Dari', 'Assyrian Neo-Aramaic', 'Cebuano', 'Maithili', 'Waray', 'Tyap', 'Basque', 
             'Telugu', 'Czech', 'Akan', 'Makasar', 'Lamba', 'Pulaar', 'Walloon', 'Thai', 'Cantonese', 'Asturian', 
             'Ibibio', 'Tagalog', 'Bengali', 'Kashubian', 'Lingala', 'Old Church Slavonic', 'Nyamwezi', 'Venda', 
             'Gusii', 'Sidamo', 'Afro-Asiatic', 'Khotanese', 'Franco-Provençal', 'Greenlandic', 'Niuean', 'Kabiyè', 
             'Maltese', 'Lahnda', 'South Slavey', 'Yuchi', 'Sorani Kurdish', 'Nyanja', 'Macedonian', 'Chuvash', 
             'Tajik', 'Aromanian', 'Soga', 'Rwa', 'Tatar', 'Yemba', 'Neapolitan', 'Romani', 'Rundi', 'Iloko', 
             'Turkmen', 'Kyrgyz', 'Plautdietsch', 'Hakka Chinese', 'Manchu', 'Taroko', 'Luganda', 'Osage', 'Avestan', 
             'Serer', 'Amharic', 'Tuvan', 'Delaware', 'Fanti', 'Ndonga', 'Inupiaq', 'Koyra Chiini', 'Mongo', 
             'Tahitian', 'Lozi', 'Yakan', 'Filipino', 'Nzima', 'Kpelle', 'Gaelic', 'Khoekhoe', 'Kabyle', 
             'Mewari', 'Moldavian', 'Mandarin', 'Tigre', 'Zoroastrian Dari', 'Skolt Sami', 'Yupik', 'Talysh', 
             'Medumba', 'Sanskrit', 'Eastern Slovak', 'Marwari', 'Sindhi', 'Yukaghir', 'Inuktitut', 
             'Bishnupriya Manipuri', 'Tiv', 'Zuni', 'Samoan', 'Afrikaans', 'Ga', 'Marathi', 'Gothic', 'Dakota', 
             'Kalmyk', 'Sardinian', 'Rajasthani', 'Kurdish', 'Lithuanian', 'Zhuang', 'Corsican', 'Zulu', 'Papiamento', 
             'Sichuan Yi', 'Khmer', 'Xhosa', 'Kikuyu', 'Emilian-Romagnol', 'Nyoro', 'Umbundu', 'Urhobo', 'Shambala', 
             'Kuanyama', 'Minangkabau', 'Swiss German', 'Quechua', 'Slovenian', 'Pashto', 'Haida', 'Mende', 'Ossetian', 
             'Ingush', 'Mossi', 'Luyia', 'Breton', 'Crimean Tatar', 'Kalaallisut', 'Jutish', 'Southern Sotho', 'Kazakh', 
             'Chichewa', 'Tonga', 'Malay', 'Rarotongan', 'Scots', 'Polish', 'Timne', 'Avaric', 'Rohingya', 'Torwali', 
             'South Ndebele', 'Chinese', 'Nuer', 'Tibetan', 'Korean', 'Somali', 'Luba-Lulua', 'Ilocano', 'Latvian', 
             'Silesian', 'Oromo', 'Persian', 'Judeo-Persian', 'Navajo', 'Sakha', 'Swabian', 'Abkhazian', 'Scottish Gaelic', 
             'Vietnamese', 'Tongan', 'Efik', 'Kongo', 'Tupi', 'Yiddish', 'Võro', 'Ewe', 'Mari', 'Masai', 'Northern Sami', 
             'Croatian', 'Chewa', 'Hindi', 'Yao', 'Algerian Arabic', 'Gan Chinese', 'Lak', 'Coptic', 'Kimbundu', 'Nogai', 
             'Western Punjabi', 'Bikol', 'Venetian', 'Kanuri', 'Mirandese', 'Northern Sotho', 'Bantu', 'Wolaytta', 'Buginese', 
             'Cree', 'Mapudungun', 'Walser', 'Flemish', 'Malagasy', 'Ukrainian', 'Koyraboro Senni', 'Hungarian', 'Teso', 
             'Montenegrin', 'Karachay-Balkar', "Toraja-Sa'dan", 'Bislama', 'Danish', 'Jju', 'Duala', 'Spanish', 'Egyptian Arabic', 
             'Hiligaynon', 'Ainu', 'Turkish', 'French', 'Lojban', 'Sundanese', 'Slovak', 'Waray-Waray', 'Tuvaluan', 'Azerbaijani', 
             'Mongolian', 'Southwestern Tamang', 'Shan', 'Southern Sami', 'Yoruba', 'Washo', 'Wolaitta', 'Bini', 'Gagauz', 
             'Faroese', 'Tetum', 'Igbo', 'Zaza']

list_lang.sort()

for lang in list_lang:
    word_lang = lang.split(" ")
    if len(word_lang) > 1:
        data = f'{{"label": "LANG", "pattern": ['
        for i in range(len(word_lang)):
            if i != len(word_lang) - 1:
                data += f'{{"LOWER": "{word_lang[i].lower()}\"}},'
            else:
                data += f'{{"LOWER": "{word_lang[i].lower()}\"}}'
        data += ']}'
    else:
        data = f'{{"label": "LANG", "pattern": [{{"LOWER": "{lang.lower()}"}}]}}'
    print(data)

{"label": "LANG", "pattern": [{"LOWER": "abkhazian"}]}
{"label": "LANG", "pattern": [{"LOWER": "adyghe"}]}
{"label": "LANG", "pattern": [{"LOWER": "afrikaans"}]}
{"label": "LANG", "pattern": [{"LOWER": "afro-asiatic"}]}
{"label": "LANG", "pattern": [{"LOWER": "ainu"}]}
{"label": "LANG", "pattern": [{"LOWER": "akan"}]}
{"label": "LANG", "pattern": [{"LOWER": "akkadian"}]}
{"label": "LANG", "pattern": [{"LOWER": "albanian"}]}
{"label": "LANG", "pattern": [{"LOWER": "algerian"},{"LOWER": "arabic"}]}
{"label": "LANG", "pattern": [{"LOWER": "amharic"}]}
{"label": "LANG", "pattern": [{"LOWER": "angika"}]}
{"label": "LANG", "pattern": [{"LOWER": "apache"}]}
{"label": "LANG", "pattern": [{"LOWER": "arabic"}]}
{"label": "LANG", "pattern": [{"LOWER": "aramaic"}]}
{"label": "LANG", "pattern": [{"LOWER": "armenian"}]}
{"label": "LANG", "pattern": [{"LOWER": "aromanian"}]}
{"label": "LANG", "pattern": [{"LOWER": "assamese"}]}
{"label": "LANG", "pattern": [{"LOWER": "assyrian"},{"LOWER": "neo-aramai

### List all the countries, sort it based on alphabet, and generate the content of hh_country_pattern.jsonl 

In [2]:
list_countries = [
'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 
'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 
'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 
'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 
'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 
'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 
'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 
'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Timor Leste', 
'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 
'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 
'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 
'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 
'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 
'Kenya', 'Kiribati', 'North Korea', 'South Korea', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 
'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 
'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 
'Mauritania', 'Mauritius', 'Mexico', 'Micronesia', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 
'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand', 
'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestine'
'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 
'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 
'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 
'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 
'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Taiwan', 
'Tajikistan', 'Tanzania', 'Thailand', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 
'Turkmenistan', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 
'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican City', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe']

list_countries.sort()

for country in list_countries:
    word_country = country.split(" ")
    if len(word_country) > 1:
        data = f'{{"label": "COUNTRY", "pattern": ['
        for i in range(len(word_country)):
            if i != len(word_country) - 1:
                data += f'{{"LOWER": "{word_country[i].lower()}\"}},'
            else:
                data += f'{{"LOWER": "{word_country[i].lower()}\"}}'
        data += ']}'
    else:
        data = f'{{"label": "COUNTRY", "pattern": [{{"LOWER": "{country.lower()}"}}]}}'
    print(data)

{"label": "COUNTRY", "pattern": [{"LOWER": "afghanistan"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "albania"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "algeria"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "andorra"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "angola"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "antigua"},{"LOWER": "and"},{"LOWER": "barbuda"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "argentina"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "armenia"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "australia"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "austria"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "azerbaijan"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "bahamas"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "bahrain"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "bangladesh"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "barbados"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "belarus"}]}
{"label": "COUNTRY", "pattern": [{"LOWER": "belgium"

---
## 4. Code Dump
### Dump

In [ ]:
#     STOPWORDS = list(set(stopwords.words('english')))
#     cst = ["LinkedIn", "Curriculam", "Vitae", "Curriculum"]
#     tx = ' '.join([word for word in tx.split() if word not in STOPWORDS+cst])
#     profDic = {}
#     skillsList = []
#     nlp = spacy.load("en_core_web_md")
#     skills = "jz_skill_patterns.jsonl"
#     ruler = nlp.add_pipe("entity_ruler", before="ner")
#     ruler.from_disk(skills)
#     patterns = [
#         {
#             "label": "EMAIL", "pattern": [{"TEXT": {"REGEX": "[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,63}"}}]
#         }
#     ]
#     ruler.add_patterns(patterns)
#     resultnlp = nlp(tx)
#     ft = [token.text for token in resultnlp if not token.is_stop]
#     fs = " ".join(ft)
#     k=0
#     for ent in resultnlp.ents:
#         if ent.label_ == 'PERSON' and k==0:
#             profDic['PERSON'] = ent.text
#             k = k+1
#         if ent.label_ == 'EMAIL':
#             email = str(ent.text)
#             if "Email:" in email:
#                 email = email[6:]
#             profDic['EMAIL'] = email
#         if ent.label_ == 'LANGUAGE':
#             profDic['LANG'] = ent.text            
#         if ent.label_ == 'SKILL':
#             trait = str(ent.text)
#             skillsList.append(trait.upper())
#             skillsList = list(set(skillsList))
#     profDic["SKILLS"] = skillsList
#     mppat = r'(?![0-9]{5}\s[0-9]{5})\+?\d{1,3}[-.\s]?\(?\d{2,3}\)?[-.\s]?\d{2,4}[-.\s]?\d{2,4}[-.\s]?\d{2,4}'
#     profDic["MOBILE"] = re.findall(mppat, tx)
#     edupat = r'\b\s*\w+(?:\s+\w+)*\s*(?i:universit(?:y|é|a(?:s|des)?|ies)|polytechnic|institute|college|academy)\w?\s*\b'
#     profDic["EDU"] = re.findall(edupat, fs)
#     EDUCATION = [
#                 'BE','B.E.', 'B.E', 'BS', 'B.S','C.A.','c.a.','B.Com','B. Com','M. Com', 'M.Com','M. Com .',
#                 'ME', 'M.E', 'M.E.', 'MS', 'M.S',
#                 'BTECH', 'B.TECH', 'M.TECH', 'MTECH',
#                 'PHD', 'phd', 'ph.d', 'Ph.D.','MBA','mba','graduate', 'post-graduate','5 year integrated masters','masters',
#                 'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
#     ]
#     resultnlp = [sent.text.strip() for sent in resultnlp.sents]
#     edu = {}
        
#     for index, text in enumerate(resultnlp):
#         for tex in text.split():
#             tex = re.sub(r'[?|$|.|!|,]', r'', tex)
#             if tex.upper() in EDUCATION and tex not in STOPWORDS:
#                 edu[tex] = text + resultnlp[index + 1]
#                 #print(edu.keys())
    
#     pdfcollection[i] = profDic
    
# df = pd.DataFrame(pdfcollection)

# degree = ["diploma", "b.tech",  "bachelor", 'b.a.', 'b.com', 'b.e.', 'b-tech', 'b.sc.', 
#         'bba', 's.kom', "mba", 'master', 'magister', 'm.sc', 'm.m', 'm.a', 'm.si', 'm.tech', 
#         'm.kom', 'm.cs', 'prof', 'ph.d']

In [ ]:
#df

---
## 5. Future Works
### Getting work experience

### Getting keywords

---
## 6. Reminder Notes
### Notes

1. Data ethical
2. Data company click (utk pengembangan)
3. Document harus ada penjelasan input, output, proses (sederhana)
4. Evaluasi sederhana